In [ ]:
# text-to-speach
!pip install -q kokoro>=0.9.4 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1
!pip install pdfplumber
!pip install pyngrok
!pip install -q git+https://github.com/openai/whisper.git

In [7]:
import io
import os
import threading
import uuid
import tempfile
import time

from flask import Flask, request, redirect, url_for, send_file, jsonify, render_template_string
import pdfplumber
from kokoro import KPipeline
import numpy as np
import soundfile as sf
from pyngrok import ngrok

app = Flask(__name__)
app.config["SECRET_KEY"] = "replace-with-your-own-secret"


# ── CONFIGURATION ───────────────────────────────────────────────────────────

# 1) List of Gemini API keys (rotate on failure)
API_KEYS = [
    "Your api key",
    "your api key"
]

# 2) Kokoro pipeline (American English voices by default)
pipeline = KPipeline(lang_code="a")

# 3) Kokoro-82M v1.0 voice list
VOICES = [
    # American English (11F, 9M)
    "af_alloy",   "af_aoede",   "af_bella",   "af_jessica", "af_kore",
    "af_nicole",  "af_nova",    "af_river",   "af_sarah",   "af_sky",
    "am_adam",    "am_echo",    "am_eric",    "am_fenrir",  "am_liam",
    "am_michael", "am_onyx",    "am_puck",
    # British English (4F, 4M)
    "bf_alice",   "bf_emma",    "bf_isabella","bf_lily",
    "bm_daniel",  "bm_fable",   "bm_george",  "bm_lewis",
    # French (1F)
    "ff_siwis",
    # Hindi (2F, 2M)
    "hf_alpha",   "hf_beta",    "hm_omega",   "hm_psi",
    # Japanese (4F, 1M)
    "jf_alpha",
    # Mandarin Chinese (4F, 4M)
    "zf_xiaobei", "zf_xiaoni",  "zf_xiaoxiao","zf_xiaoyi",
    "zm_yunjian", "zm_yunxi",   "zm_yunxia",  "zm_yunyang",
]

# In-memory storage for documents & TTS jobs
DOCUMENTS = {}  # doc_id → {"filename": str, "pages": [str, ...]}
TTS_JOBS   = {}  # job_id → {"doc_id": str, "voice": str, "audio_path": str, "duration": float, "status": str, "line_timings": [[{start,end,text},...], ...]}


# ── UTILITIES ─────────────────────────────────────────────────────────────────

def extract_text_and_tables(pdf_path):
    """
    Extract each page's text (excluding tables), insert placeholders for tables,
    then return:
      - pages_text: list of strings (one per page, with placeholders only)
      - tables: list of (placeholder, table_data) for all pages
    """
    pages_text = []
    all_tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 1) find all tables on that page
            table_objs  = page.find_tables()
            table_bboxes = [tbl.bbox for tbl in table_objs]

            # 2) extract all words, but filter out those inside any table bbox
            all_words      = page.extract_words()
            non_table_words = []
            for w in all_words:
                x0, x1, top, bottom = w["x0"], w["x1"], w["top"], w["bottom"]
                inside = False
                for (tx0, ttop, tx1, tbottom) in table_bboxes:
                    if x0 >= tx0 and x1 <= tx1 and top >= ttop and bottom <= tbottom:
                        inside = True
                        break
                if not inside:
                    non_table_words.append(w)

            # 3) group non-table words into lines by similar 'top'
            non_table_words.sort(key=lambda w: (round(w["top"]), w["x0"]))
            lines = []
            current_line = []
            current_top  = None
            for w in non_table_words:
                w_top = w["top"]
                if current_top is None:
                    current_top = w_top
                    current_line = [w]
                elif abs(w_top - current_top) < 3:
                    current_line.append(w)
                else:
                    line_text = " ".join(word["text"] for word in current_line)
                    line_top  = sum(word["top"] for word in current_line) / len(current_line)
                    lines.append((line_text, line_top))
                    current_line = [w]
                    current_top = w_top
            if current_line:
                line_text = " ".join(word["text"] for word in current_line)
                line_top  = sum(word["top"] for word in current_line) / len(current_line)
                lines.append((line_text, line_top))

            # 4) insert placeholders for each table at the correct line position
            for tbl_idx, tbl_obj in enumerate(table_objs):
                placeholder = f"[[TABLE_PAGE_{page.page_number}_IDX_{tbl_idx}]]"
                table_data   = tbl_obj.extract()
                all_tables.append((placeholder, table_data))

                table_top = tbl_obj.bbox[1]
                insert_idx = None
                for i, (_, line_top) in enumerate(lines):
                    if line_top > table_top:
                        insert_idx = i
                        break
                if insert_idx is None:
                    lines.append((placeholder, table_top))
                else:
                    lines.insert(insert_idx, (placeholder, table_top))

            # Build final page text (just joining lines, no “[PAGE X]” header)
            page_text = "\n".join(line for line, _ in lines)
            pages_text.append(page_text)

    return pages_text, all_tables


def convert_tables_with_gemini(tables):
    """
    Given list of (placeholder, table_data), send each to Gemini to get back a paragraph.
    Uses API_KEYS: on failure, waits briefly, switches to next key, retries.
    Returns list of (placeholder, paragraph_text). If all retries fail, paragraph_text = "".
    """
    conversions = []

    for placeholder, table_data in tables:
        # Build a markdown-like string so Gemini can detect columns
        markdown_table = "\n".join(" | ".join(map(str, row)) for row in table_data)
        prompt = f"""
Convert the following table into one smooth English paragraph.
Do NOT include ANY special symbols (#, *, |, etc.).
Just describe the same information in normal sentences:

{markdown_table}
"""

        paragraph = ""
        success   = False

        # Try each API key in turn, with a small delay between attempts
        for key in API_KEYS:
            client = genai.Client(api_key=key)
            try:
                resp      = client.models.generate_content(
                    model="gemini-2.0-flash",
                    contents=prompt
                )
                paragraph = resp.text.strip()
                success   = True
                break
            except Exception:
                time.sleep(2)
                continue

        if not success:
            paragraph = ""

        conversions.append((placeholder, paragraph))

    return conversions


def replace_placeholders_in_pages(pages, conversions):
    """
    For each page's text, replace any placeholder with its converted paragraph.
    """
    out_pages = []
    for page_text in pages:
        txt = page_text
        for placeholder, paragraph in conversions:
            txt = txt.replace(placeholder, paragraph)
        out_pages.append(txt)
    return out_pages


def tts_generate_full_audio(doc_id, voice):
    """
    Generate full-document TTS for DOCUMENTS[doc_id].
    Write WAV to a temp file. Return (wav_path, duration_seconds).
    """
    pages      = DOCUMENTS[doc_id]["pages"]
    full_script = "\n\n".join(pages)

    generator = pipeline(
        full_script,
        voice=voice,
        speed=1,
        split_pattern=None
    )
    chunks = []
    for (_gs, _ps, audio_chunk) in generator:
        chunks.append(audio_chunk)

    if not chunks:
        return None, 0.0

    full_audio = np.concatenate(chunks, axis=0)
    tmp_wav    = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(tmp_wav.name, full_audio, 24000)
    duration = len(full_audio) / 24000.0
    return tmp_wav.name, duration


# ── FLASK ROUTES ─────────────────────────────────────────────────────────────────

@app.route("/", methods=["GET", "POST"])
def index():
    """
    GET: Show upload form + library sidebar.
    POST: Save PDF, extract text/tables, convert tables via Gemini, store pages → redirect to /reader/<doc_id>
    """
    if request.method == "POST":
        f = request.files.get("pdf_file")
        if not f or not f.filename.lower().endswith(".pdf"):
            return redirect(request.url)

        doc_id = str(uuid.uuid4())
        filename = f.filename
        tmp_pdf = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
        f.save(tmp_pdf.name)

        # STEP A: extract pages + table placeholders (no “[PAGE X]” headers)
        pages, tables = extract_text_and_tables(tmp_pdf.name)

        # STEP B: convert tables with Gemini (retry + rotate keys)
        conversions = convert_tables_with_gemini(tables)

        # STEP C: replace placeholders
        clean_pages = replace_placeholders_in_pages(pages, conversions)

        DOCUMENTS[doc_id] = {
            "filename": filename,
            "pages": clean_pages,
        }
        return redirect(url_for("reader", doc_id=doc_id))

    # GET: render upload + library
    library = [
        {"doc_id": d, "filename": DOCUMENTS[d]["filename"]}
        for d in DOCUMENTS
    ]
    return render_template_string(INDEX_HTML, library=library, voices=VOICES)


@app.route("/reader/<doc_id>")
def reader(doc_id):
    """
    Render the "modern reader" UI for the selected document.
    """
    if doc_id not in DOCUMENTS:
        return redirect(url_for("index"))

    pages = DOCUMENTS[doc_id]["pages"]
    latest_tts_job = None
    latest_job_id  = None
    line_timings   = None

    # ── FIND THE MOST RECENT READY TTS JOB FOR THIS doc_id ─────────────────────────────
    for job_id, job in reversed(list(TTS_JOBS.items())):
        if job["doc_id"] == doc_id and job.get("status") == "ready":
            latest_tts_job = job
            latest_job_id  = job_id
            line_timings   = job.get("line_timings", None)
            break

    return render_template_string(
        READER_HTML,
        doc_id=doc_id,
        filename=DOCUMENTS[doc_id]["filename"],
        pages=pages,
        voices=VOICES,
        line_timings=line_timings,
        latest_tts_job=latest_tts_job,
        latest_job_id=latest_job_id
    )


@app.route("/generate_audio", methods=["POST"])
def generate_audio():
    """
    AJAX endpoint: given doc_id & voice, start a background TTS job.
    Returns { job_id } immediately. Client will poll /tts_status/<job_id>.
    """
    data   = request.json or {}
    doc_id = data.get("doc_id")
    voice  = data.get("voice")
    if not doc_id or not voice:
        return jsonify({"error": "doc_id and voice are required"}), 400
    if doc_id not in DOCUMENTS:
        return jsonify({"error": "invalid doc_id"}), 404

    job_id = str(uuid.uuid4())
    TTS_JOBS[job_id] = {
        "doc_id": doc_id,
        "voice": voice,
        "audio_path": None,
        "duration": 0.0,
        "status": "pending",
        "line_timings": None
    }

    def worker():
        # 1) Generate TTS‐WAV
        wav_path, duration = tts_generate_full_audio(doc_id, voice)
        if not wav_path:
            TTS_JOBS[job_id]["status"] = "error"
            return

        # 2) Compute character-based weights for every line
        pages = DOCUMENTS[doc_id]["pages"]
        pages_lines = [page_text.split("\n") for page_text in pages]

        # Flatten into a list of (page_idx, line_idx, line_text)
        flat_lines = []
        for pi, page_list in enumerate(pages_lines):
            for li, line_text in enumerate(page_list):
                flat_lines.append((pi, li, line_text))

        # Count total characters (including spaces, punctuation)
        total_chars = sum(len(line_text) for (_, _, line_text) in flat_lines)
        if total_chars == 0:
            total_chars = 1  # avoid division by zero

        # Build line_timings flat: each line gets a time slice proportional to its length
        flat_timings = []
        cum_chars = 0
        for (_, _, line_text) in flat_lines:
            length = len(line_text)
            start_frac = cum_chars / total_chars
            cum_chars += length
            end_frac = cum_chars / total_chars
            start_ts = round(start_frac * duration, 3)
            end_ts   = round(end_frac   * duration,   3)
            flat_timings.append({
                "start": start_ts,
                "end":   end_ts,
                "text":  line_text
            })

        # Re‐group into pages
        line_timings = []
        ptr = 0
        for page_list in pages_lines:
            page_timings = []
            for _ in page_list:
                page_timings.append(flat_timings[ptr])
                ptr += 1
            line_timings.append(page_timings)

        # 3) Save into TTS_JOBS
        TTS_JOBS[job_id]["line_timings"] = line_timings
        TTS_JOBS[job_id]["audio_path"]   = wav_path
        TTS_JOBS[job_id]["duration"]     = duration
        TTS_JOBS[job_id]["status"]       = "ready"

    thread = threading.Thread(target=worker, daemon=True)
    thread.start()

    return jsonify({"job_id": job_id})


@app.route("/tts_status/<job_id>")
def tts_status(job_id):
    """
    Poll this to see if a TTS job is ready. Returns JSON with "status": pending/ready/error
    and, if ready, an "audio_url" to retrieve the WAV.
    """
    job = TTS_JOBS.get(job_id)
    if not job:
        return jsonify({"error": "not found"}), 404

    if job["status"] == "ready":
        return jsonify({
            "status":    "ready",
            "audio_url": url_for("get_audio", job_id=job_id)
        })
    else:
        return jsonify({"status": job["status"]})


@app.route("/get_audio/<job_id>")
def get_audio(job_id):
    """
    Streams back the generated WAV file for a completed TTS job.
    """
    job = TTS_JOBS.get(job_id)
    if not job or job["status"] != "ready":
        return redirect(url_for("index"))
    return send_file(job["audio_path"], mimetype="audio/wav")


@app.route("/preview_audio", methods=["POST"])
def preview_audio():
    """
    Generate a short preview (first 100 chars) for doc_id & voice.
    Returns a small WAV blob to play inline.
    """
    data   = request.json or {}
    doc_id = data.get("doc_id")
    voice  = data.get("voice")
    if not doc_id or not voice:
        return jsonify({"error": "doc_id and voice are required"}), 400
    if doc_id not in DOCUMENTS:
        return jsonify({"error": "invalid doc_id"}), 404

    pages  = DOCUMENTS[doc_id]["pages"]
    snippet = " ".join(pages)
    snippet = snippet[:100] + ("…" if len(snippet) > 100 else "")

    generator = pipeline(snippet, voice=voice, speed=1, split_pattern=None)
    chunks = [audio for (_gs, _ps, audio) in generator]
    if not chunks:
        return jsonify({"error": "TTS failed"}), 500

    audio_data = np.concatenate(chunks, axis=0)
    buf = io.BytesIO()
    sf.write(buf, audio_data, 24000, format="WAV")
    buf.seek(0)
    return send_file(buf, mimetype="audio/wav")


# Inject DOCUMENTS and TTS_JOBS into Jinja2 context
@app.context_processor
def inject_data():
    return dict(DOCUMENTS=DOCUMENTS, TTS_JOBS=TTS_JOBS)


# ── HTML TEMPLATES ───────────────────────────────────────────────────────────────

INDEX_HTML = """
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>PDF→TTS Library</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
  <style>
    :root {
      --primary: #2563eb;
      --accent: #60a5fa;
      --bg: #f9fafb;
      --card-bg: #fff;
      --sidebar-bg: #f3f4f6;
      --sidebar-icon: #94a3b8;
      --text-main: #222;
      --text-light: #64748b;
      --shadow: 0 4px 24px rgba(30,64,175,0.07);
      --radius: 18px;
    }
    html, body { height: 100%; }
    body {
      font-family: 'Inter', Arial, sans-serif;
      margin: 0; padding: 0;
      background: var(--bg);
      min-height: 100vh;
      color: var(--text-main);
    }
    .sidebar {
      position: fixed; top: 0; left: 0; width: 72px; height: 100vh;
      background: var(--sidebar-bg); display: flex; flex-direction: column;
      align-items: center; padding: 1.5rem 0; z-index: 1000;
      box-shadow: 2px 0 16px rgba(30,64,175,0.04);
      border-top-right-radius: var(--radius);
      border-bottom-right-radius: var(--radius);
      gap: 2rem;
    }
    .sidebar .icon-btn {
      background: none; border: none; color: var(--sidebar-icon);
      font-size: 1.7rem; margin: 0.5rem 0; cursor: pointer;
      transition: color 0.2s, background 0.2s;
      border-radius: 50%; padding: 0.7rem;
      display: flex; align-items: center; justify-content: center;
    }
    .sidebar .icon-btn.active, .sidebar .icon-btn:hover {
      color: var(--primary); background: #e0e7ef;
    }
    .sidebar .spacer { flex: 1; }
    .main {
      margin-left: 72px; padding: 3.5rem 2rem 6rem 2rem;
      min-height: 100vh; box-sizing: border-box;
      display: flex; flex-direction: column; align-items: center;
    }
    .upload-card {
      background: var(--card-bg); border-radius: var(--radius); box-shadow: var(--shadow);
      padding: 2.5rem 2rem; max-width: 420px; margin: 3rem auto 0 auto;
      display: flex; flex-direction: column; align-items: center;
      border: 1px solid #e5e7eb;
    }
    .upload-card h1 {
      font-size: 2rem; font-weight: 700; margin-bottom: 1.2rem; color: var(--primary);
    }
    .upload-card form {
      width: 100%; display: flex; flex-direction: column; gap: 1.2rem;
    }
    .upload-card input[type='file'] {
      padding: 0.7rem; border-radius: 8px; border: 1px solid #d1d5db;
      background: #f3f4f6; font-size: 1rem;
    }
    .upload-card button {
      background: linear-gradient(90deg, var(--primary) 0%, var(--accent) 100%);
      color: #fff; border: none; border-radius: 8px; padding: 0.8rem 1.5rem;
      font-size: 1.1rem; font-weight: 600; cursor: pointer;
      box-shadow: 0 2px 8px rgba(30,64,175,0.08);
      transition: background 0.2s;
      display: flex; align-items: center; justify-content: center;
      gap: 0.6rem;
    }
    .upload-card button:hover {
      background: linear-gradient(90deg, var(--primary) 0%, #1e40af 100%);
    }
    .library-list {
      margin-top: 2.5rem; width: 100%;
    }
    .library-list h3 {
      font-size: 1.1rem; font-weight: 600; margin-bottom: 0.7rem;
      color: var(--primary);
    }
    .library-list .empty {
      color: #b2becd; font-size: 1rem; margin-top: 1rem;
    }
    .library-list a {
      display: block; color: var(--primary); text-decoration: none;
      padding: 0.7rem 1rem; border-radius: 10px; margin: 0.2rem 0;
      background: #f1f5f9; font-weight: 500; transition: background 0.2s;
    }
    .library-list a:hover { background: #e0e7ef; }
    .footer {
      width: 100%; text-align: center; color: var(--text-light); font-size: 0.95rem;
      margin-top: 3rem; padding: 1.5rem 0 0.5rem 0;
    }
    .loader {
      display: none; margin: 1.5rem auto 0 auto; width: 48px; height: 48px;
      border: 5px solid #e5e7eb; border-top: 5px solid var(--primary);
      border-radius: 50%; animation: spin 1s linear infinite;
    }
    .loader.active { display: block; }
    @keyframes spin {
      0% { transform: rotate(0deg); }
      100% { transform: rotate(360deg); }
    }
    @media (max-width: 700px) {
      .sidebar { width: 56px; }
      .main { margin-left: 56px; padding: 1.2rem 0.5rem 6rem 0.5rem; }
      .upload-card { padding: 1.2rem; }
    }
    @media (max-width: 500px) {
      .main { margin-left: 0; padding: 0.5rem 0.2rem 6rem 0.2rem; }
      .sidebar { display: none; }
    }
  </style>
</head>
<body>
  <div class="sidebar">
    <button class="icon-btn active" title="Upload">⬆️</button>
    <div class="spacer"></div>
    <button class="icon-btn" title="Library">📚</button>
  </div>
  <div class="main">
    <div class="upload-card">
      <h1>Upload a PDF</h1>
      <form method="post" enctype="multipart/form-data" onsubmit="showLoader()">
        <input type="file" name="pdf_file" accept=".pdf" required>
        <button type="submit"><span>⬆️</span>Upload &amp; Process</button>
      </form>
      <div class="loader" id="loader"></div>
    </div>

    <div class="library-list">
      <h3>Your Library</h3>
      {% if library %}
        {% for doc in library %}
          <a href="{{ url_for('reader', doc_id=doc.doc_id) }}">📄 {{ doc.filename }}</a>
        {% endfor %}
      {% else %}
        <div class="empty">No PDFs uploaded yet.</div>
      {% endif %}
    </div>

    <div class="footer">&copy; 2024 PDF→TTS Library</div>
  </div>

  <script>
    function showLoader() {
      document.getElementById('loader').classList.add('active');
    }
  </script>
</body>
</html>
"""

READER_HTML = """
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Reader: {{ filename }}</title>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
  <style>
    :root {
      --primary: #2563eb;
      --accent: #60a5fa;
      --bg: #f9fafb;
      --card-bg: #fff;
      --sidebar-bg: #f3f4f6;
      --sidebar-icon: #94a3b8;
      --text-main: #222;
      --text-light: #64748b;
      --shadow: 0 4px 24px rgba(30,64,175,0.07);
      --radius: 18px;
      --highlight: #e0e7ef;
    }
    html, body { height: 100%; margin:0; padding:0; }
    body {
      font-family: 'Inter', Arial, sans-serif;
      background: var(--bg);
      color: var(--text-main);
      min-height: 100vh;
    }
    .sidebar {
      position: fixed; top: 0; left: 0; width: 72px; height: 100vh;
      background: var(--sidebar-bg); display: flex; flex-direction: column;
      align-items: center; padding: 1.5rem 0; z-index: 1000;
      box-shadow: 2px 0 16px rgba(30,64,175,0.04);
      border-top-right-radius: var(--radius);
      border-bottom-right-radius: var(--radius);
      gap: 2rem;
    }
    .sidebar .icon-btn {
      background: none; border: none; color: var(--sidebar-icon);
      font-size: 1.7rem; margin: 0.5rem 0; cursor: pointer;
      transition: color 0.2s, background 0.2s;
      border-radius: 50%; padding: 0.7rem;
      display: flex; align-items: center; justify-content: center;
    }
    .sidebar .icon-btn.active, .sidebar .icon-btn:hover {
      color: var(--primary); background: #e0e7ef;
    }
    .sidebar .spacer { flex: 1; }
    .main {
      margin-left: 72px; padding: 3.5rem 2rem 6rem 2rem;
      box-sizing: border-box; display: flex; flex-direction: column; align-items: center;
    }
    .page-card {
      background: var(--card-bg); border-radius: var(--radius); box-shadow: var(--shadow);
      margin-bottom: 2.2rem; padding: 2rem 1.5rem 1.5rem 1.5rem;
      transition: box-shadow 0.2s, background 0.2s;
      position: relative; width: 100%; max-width: 700px;
      border: 1px solid #e5e7eb;
    }
    .page-card.highlighted {
      background: #e3f0ff;
      box-shadow: 0 4px 24px rgba(41,128,185,0.13);
    }
    .page-card h2 {
      margin-top: 0; font-size: 1.1rem; color: var(--primary); font-weight: 700;
    }
    .page-card .line {
      display: block; padding: 0.1rem 0.2rem; border-radius: 6px;
      transition: background 0.2s;
      white-space: pre-wrap; font-family: 'Inter', Arial, sans-serif; font-size: 1.05rem; color: var(--text-main);
    }
    .page-card .line.highlight {
      background: var(--highlight);
      color: var(--primary);
      font-weight: 600;
    }
    .footer {
      width: 100%; text-align: center; color: var(--text-light); font-size: 0.95rem;
      margin-top: 3rem; padding: 1.5rem 0 0.5rem 0;
    }
    .loader {
      display: none; margin: 1.5rem auto 0 auto; width: 48px; height: 48px;
      border: 5px solid #e5e7eb; border-top: 5px solid var(--primary);
      border-radius: 50%; animation: spin 1s linear infinite;
    }
    .loader.active { display: block; }
    @keyframes spin {
      0% { transform: rotate(0deg); }
      100% { transform: rotate(360deg); }
    }
    .audio-dock {
      position: fixed; left: 0; bottom: 0; width: 100vw; background: var(--card-bg);
      box-shadow: 0 -2px 24px rgba(30,64,175,0.07);
      border-top: 1px solid #e5e7eb;
      display: flex; align-items: center; justify-content: center;
      padding: 1.2rem 2rem; z-index: 2000;
      gap: 1.5rem;
    }
    .audio-dock audio {
      width: 100%; max-width: 600px;
      background: #f3f4f6; border-radius: 8px;
    }
    .audio-dock .status {
      color: var(--primary); font-size: 1rem; margin-left: 1rem;
    }
    @media (max-width: 700px) {
      .sidebar { width: 56px; }
      .main { margin-left: 56px; padding: 1.2rem 0.5rem 6rem 0.5rem; }
      .page-card { padding: 1.2rem; }
      .audio-dock { padding: 0.7rem 0.5rem; }
    }
    @media (max-width: 500px) {
      .main { margin-left: 0; padding: 0.5rem 0.2rem 6rem 0.2rem; }
      .sidebar { display: none; }
      .audio-dock { flex-direction: column; gap: 0.7rem; }
    }
  </style>
</head>
<body>
  <div class="sidebar">
    <button class="icon-btn" title="Back" onclick="window.location='{{ url_for('index') }}'">⬅️</button>
    <div class="spacer"></div>
    <button class="icon-btn active" title="Reader">📄</button>
  </div>
  <div class="main">
    <h1 style="font-size:2rem; font-weight:700; color:var(--primary); margin-bottom:2.2rem;">
      Reading: {{ filename }}
    </h1>

    <div id="pagesContainer" style="width:100%; max-width:700px;">
      {% if line_timings %}
        {# LOOP PAGE BY PAGE USING JINJA2'S loop.index #}
        {% for page_lines in line_timings %}
          {% set page_idx = loop.index %}
          <div class="page-card" id="page-{{ page_idx }}">
            <h2>Page {{ page_idx }}</h2>
            {% for ln in page_lines %}
              <span class="line" data-start="{{ ln.start }}" data-end="{{ ln.end }}">
                {{ ln.text }}
              </span>
            {% endfor %}
          </div>
        {% endfor %}
      {% else %}
        {# If no line_timings yet, just show raw PDF text (no data-start/data-end) #}
        {% for text in pages %}
          {% set page_idx = loop.index %}
          <div class="page-card" id="page-{{ page_idx }}">
            <h2>Page {{ page_idx }}</h2>
            <pre style="white-space: pre-wrap; font-family: 'Inter', Arial, sans-serif; font-size: 1.05rem; color: var(--text-main); margin: 0.7rem 0 0 0; background: none; border: none;">
              {% for line in text.split('\n') %}
                <span class="line">{{ line }}</span>
              {% endfor %}
            </pre>
          </div>
        {% endfor %}
      {% endif %}
    </div>

    <div class="footer">&copy; 2024 PDF→TTS Library</div>
    <div class="loader" id="loader"></div>
  </div>

  <div class="audio-dock">
    <button onclick="startTTS()" id="startBtn" style="margin-right:1rem;">▶️ Start</button>

    <audio id="audioPlayer" controls style="width:100%; max-width:600px;"
      {% if latest_tts_job and latest_job_id and latest_tts_job.status == 'ready' %}
        src="{{ url_for('get_audio', job_id=latest_job_id) }}"
      {% endif %}
    ></audio>

    <div class="status" id="status"></div>

    <select id="voiceSelect" style="margin-left:1rem;">
      {% for v in voices %}
        <option value="{{ v }}">{{ v }}</option>
      {% endfor %}
    </select>

    <button onclick="previewAudio()">🔊 Preview</button>
  </div>

  <script>
    const docId = "{{ doc_id }}";
    let audioPlayer = document.getElementById("audioPlayer");
    let statusDiv   = document.getElementById("status");
    let loader      = document.getElementById("loader");
    let voiceSelect = document.getElementById("voiceSelect");

    function showLoader() { loader.classList.add('active'); }
    function hideLoader() { loader.classList.remove('active'); }

    function startTTS() {
      statusDiv.innerText = "Generating audio…";
      showLoader();
      const voice = voiceSelect.value;
      fetch("/generate_audio", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ doc_id: docId, voice: voice })
      })
      .then(res => res.json())
      .then(data => {
        if (data.job_id) {
          pollStatus(data.job_id);
        } else {
          statusDiv.innerText = "Error starting TTS.";
          hideLoader();
        }
      });
    }

    function pollStatus(jobId) {
      fetch("/tts_status/" + jobId)
        .then(res => res.json())
        .then(data => {
          if (data.status === "pending") {
            statusDiv.innerText = "Still processing…";
            setTimeout(() => pollStatus(jobId), 1200);
          } else if (data.status === "ready") {
            statusDiv.innerText = "Done. Reloading…";
            window.location = "/reader/" + docId;
          } else {
            statusDiv.innerText = "TTS failed.";
            hideLoader();
          }
        });
    }

    function previewAudio() {
      const voice = voiceSelect.value;
      statusDiv.innerText = "Generating preview…";
      showLoader();
      fetch("/preview_audio", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ doc_id: docId, voice: voice })
      })
      .then(res => res.blob())
      .then(blob => {
        const url = URL.createObjectURL(blob);
        const previewPlayer = new Audio(url);
        previewPlayer.play();
        statusDiv.innerText = "Preview playing…";
        previewPlayer.onended = () => {
          statusDiv.innerText = "";
          hideLoader();
        };
      })
      .catch(() => {
        statusDiv.innerText = "Preview failed.";
        hideLoader();
      });
    }

    // ── HIGHLIGHT + AUTOSCROLL ────────────────────────────────────────────────────
    window.onload = function() {
      // If an audio src is already present (i.e. job is ready), start playing
      if (audioPlayer.src) {
        audioPlayer.play();
      }
    };

    audioPlayer.ontimeupdate = function() {
      const lines = Array.from(document.querySelectorAll('.line[data-start][data-end]'));
      if (!lines.length || !audioPlayer.duration) return;
      let t = audioPlayer.currentTime;
      let found = false;
      for (let i = 0; i < lines.length; i++) {
        let start = parseFloat(lines[i].getAttribute('data-start'));
        let end   = parseFloat(lines[i].getAttribute('data-end'));
        if (t >= start && t < end) {
          lines.forEach(l => l.classList.remove('highlight'));
          lines[i].classList.add('highlight');
          // Auto-scroll so that this line is roughly centered
          lines[i].scrollIntoView({ behavior: "smooth", block: "center" });
          found = true;
          break;
        }
      }
      if (!found) {
        lines.forEach(l => l.classList.remove('highlight'));
      }
    };
  </script>
</body>
</html>
"""


# ── ENTRY POINT ─────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    # Start ngrok tunnel on port 5000
    port      = 5000
    public_url = ngrok.connect(port).public_url
    print(f" * ngrok tunnel running at: {public_url}")
    print(" * Press CTRL+C to quit.")
    app.run(host="0.0.0.0", port=port)


 * ngrok tunnel running at: https://6cb6-34-16-249-112.ngrok-free.app
 * Press CTRL+C to quit.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:15:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:15:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:15:26] "POST / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:15:27] "GET /reader/54f68122-b528-47b3-b860-d93604e93cc1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:16:48] "POST /generate_audio HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:16:49] "GET /tts_status/686b246d-5f24-4b4a-acdd-c9ff2bc1a02d HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/2025 23:16:51] "GET /tts_status/686b246d-5f24-4b4a-acdd-c9ff2bc1a02d HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Jun/20